# News Headline Rewriter/Summarizer

This notebook demonstrates the fine-tuned T5-small model for generating news headlines with LangChain integration for flexible prompting styles.


## 1. Setup and Imports


In [ ]:
# import torch
# from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
# from langchain.llms import HuggingFacePipeline
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
# import warnings
# warnings.filterwarnings('ignore')

# # Check device
# device = 0 if torch.cuda.is_available() else -1
# print(f"Using device: {'CUDA' if device == 0 else 'CPU'}")


# WORKING CODE:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline  # community namespace
from langchain_core.prompts import PromptTemplate

## 2. Load Fine-tuned Model

Load the T5-small model that was fine-tuned in `train_model.ipynb`.


In [ ]:
# Path to fine-tuned model
model_path = "./models/t5-small-headlines-final"

# Load model and tokenizer
print("Loading fine-tuned model...")
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Create summarization pipeline
summarization_pipeline = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device=device,
    max_length=128,
    min_length=10,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

print("Model loaded successfully!")


## 3. Basic Headline Generation

Test the model with basic summarization before adding LangChain.


In [ ]:
def generate_headline(article_text, max_length=128, num_beams=4):
    """Generate a headline from an article using the fine-tuned model."""
    result = summarization_pipeline(
        article_text,
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True
    )
    return result[0]['summary_text']

# Test with a sample article
sample_article = """
Breaking news from Washington: The Senate has passed a landmark climate bill 
that aims to reduce carbon emissions by 40% over the next decade. The bill, 
which received bipartisan support, includes significant investments in renewable 
energy infrastructure and tax incentives for green technology companies. 
Environmental groups have praised the legislation as a major step forward in 
combating climate change, while some industry leaders express concerns about 
the economic impact of the new regulations.
"""

print("Sample Article:")
print(sample_article)
print("\n" + "="*70)
print("\nGenerated Headline:")
headline = generate_headline(sample_article)
print(headline)


## 4. LangChain Integration

Integrate LangChain to enable flexible prompting styles like "make it more exciting" or "make it more formal".


In [ ]:
# Wrap the summarization pipeline in LangChain
llm = HuggingFacePipeline(pipeline=summarization_pipeline)

print("LangChain integration ready!")


## 5. Create Different Headline Styles

Define functions to generate headlines in different styles using creative temperature and sampling parameters.


In [ ]:
class HeadlineStyler:
    """Generate headlines in different styles."""
    
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
    
    def _generate(self, text, temperature=0.7, top_p=0.9, max_length=64):
        """Internal generation method with custom parameters."""
        pipe = pipeline(
            "summarization",
            model=self.model,
            tokenizer=self.tokenizer,
            device=self.device,
            max_length=max_length,
            min_length=5,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            num_return_sequences=1
        )
        result = pipe(text)
        return result[0]['summary_text']
    
    def exciting(self, article):
        """Generate an exciting, attention-grabbing headline."""
        # Higher temperature for more creative/exciting outputs
        return self._generate(article, temperature=0.9, top_p=0.95, max_length=64)
    
    def formal(self, article):
        """Generate a formal, professional headline."""
        # Lower temperature for more conservative outputs
        return self._generate(article, temperature=0.5, top_p=0.8, max_length=64)
    
    def short(self, article):
        """Generate a short, concise headline."""
        # Shorter max length
        return self._generate(article, temperature=0.7, top_p=0.9, max_length=32)
    
    def detailed(self, article):
        """Generate a longer, more detailed headline."""
        # Longer max length
        return self._generate(article, temperature=0.7, top_p=0.9, max_length=128)
    
    def neutral(self, article):
        """Generate a balanced, neutral headline."""
        # Balanced parameters
        return self._generate(article, temperature=0.7, top_p=0.9, max_length=64)

# Initialize the styler
styler = HeadlineStyler(model, tokenizer, device)
print("Headline Styler initialized!")


## 6. Demo: Different Headline Styles

Let's test the different styles on various news articles.


In [ ]:
def demo_styles(article, title="News Article"):
    """Demonstrate all headline styles for a given article."""
    print(f"\n{'='*80}")
    print(f"  {title}")
    print(f"{'='*80}\n")
    print("Original Article:")
    print(article[:300] + "..." if len(article) > 300 else article)
    print(f"\n{'-'*80}\n")
    
    print("🎯 NEUTRAL HEADLINE:")
    print(f"   {styler.neutral(article)}\n")
    
    print("🚀 EXCITING HEADLINE:")
    print(f"   {styler.exciting(article)}\n")
    
    print("📰 FORMAL HEADLINE:")
    print(f"   {styler.formal(article)}\n")
    
    print("⚡ SHORT HEADLINE:")
    print(f"   {styler.short(article)}\n")
    
    print("📝 DETAILED HEADLINE:")
    print(f"   {styler.detailed(article)}\n")

# Test Article 1: Technology
tech_article = """
Apple Inc. announced today the launch of its latest iPhone model, featuring 
revolutionary AI capabilities and a groundbreaking camera system. The new device 
includes an advanced neural engine that can process complex machine learning tasks 
directly on the phone, eliminating the need for cloud processing. Industry analysts 
predict the new iPhone will set sales records, with pre-orders already exceeding 
expectations. The starting price is set at $999, with premium models reaching $1,499.
"""

demo_styles(tech_article, "Technology News")


In [ ]:
# Test Article 2: Sports
sports_article = """
In a stunning upset at the championship finals, the underdog team defeated the 
reigning champions 3-2 in overtime. The rookie player scored the winning goal 
in the final seconds, sending fans into a frenzy. This marks the first championship 
victory for the franchise in over 50 years. The coach credited the team's dedication 
and training regimen for their success. Celebrations erupted in the city as thousands 
of fans took to the streets to celebrate the historic win.
"""

demo_styles(sports_article, "Sports News")


In [ ]:
# Test Article 3: Science
science_article = """
Researchers at MIT have developed a new material that can change its properties 
in response to temperature, potentially revolutionizing construction and manufacturing. 
The material, which combines polymers with nanoscale structures, can become rigid 
or flexible depending on environmental conditions. Scientists believe this breakthrough 
could lead to self-healing buildings and adaptive clothing. The research, published 
in Nature Materials, has attracted significant interest from major technology companies 
looking to commercialize the discovery.
"""

demo_styles(science_article, "Science News")


## 7. Interactive Headline Generator

Create your own headlines by entering custom articles below!


In [ ]:
def generate_custom_headline(article_text, style="neutral"):
    """
    Generate a headline with a specific style.
    
    Args:
        article_text: The news article text
        style: One of 'neutral', 'exciting', 'formal', 'short', 'detailed'
    """
    style_map = {
        'neutral': styler.neutral,
        'exciting': styler.exciting,
        'formal': styler.formal,
        'short': styler.short,
        'detailed': styler.detailed
    }
    
    if style not in style_map:
        print(f"Unknown style: {style}. Using 'neutral' instead.")
        style = 'neutral'
    
    return style_map[style](article_text)

# Example usage
print("Interactive Headline Generator Ready!")
print("\nAvailable styles: neutral, exciting, formal, short, detailed")
print("\nExample:")
custom_article = """
The local city council approved a new public park project that will transform 
an abandoned industrial site into a green space with playgrounds, walking trails, 
and a community garden. The project is expected to be completed within two years 
and will cost $5 million, funded through a combination of city budget and private donations.
"""

print(f"\nArticle: {custom_article[:200]}...")
print(f"\nExciting Style: {generate_custom_headline(custom_article, 'exciting')}")
print(f"Formal Style: {generate_custom_headline(custom_article, 'formal')}")
print(f"Short Style: {generate_custom_headline(custom_article, 'short')}")


## 8. Comparison with Pre-trained Model

Let's compare our fine-tuned model with the original T5-small to see the improvement.


In [ ]:
# Load original T5-small for comparison
print("Loading original T5-small model for comparison...")
original_model = T5ForConditionalGeneration.from_pretrained("t5-small")
original_tokenizer = T5Tokenizer.from_pretrained("t5-small")

original_pipeline = pipeline(
    "summarization",
    model=original_model,
    tokenizer=original_tokenizer,
    device=device,
    max_length=128,
    min_length=10
)

def compare_models(article):
    """Compare headlines from original and fine-tuned models."""
    print("\n" + "="*80)
    print("Article:", article[:200] + "...")
    print("="*80 + "\n")
    
    original_result = original_pipeline(article, do_sample=False)
    finetuned_result = generate_headline(article)
    
    print("📘 ORIGINAL T5-SMALL:")
    print(f"   {original_result[0]['summary_text']}\n")
    
    print("✨ FINE-TUNED MODEL:")
    print(f"   {finetuned_result}\n")

# Test comparison
comparison_article = """
A major earthquake measuring 7.2 on the Richter scale struck the coastal region 
early this morning, causing widespread damage and triggering tsunami warnings. 
Emergency services are responding to multiple collapsed buildings, and authorities 
have evacuated thousands of residents from low-lying areas. The government has 
declared a state of emergency and called for international assistance.
"""

compare_models(comparison_article)

print("\n" + "💡 Note: The fine-tuned model should produce more news-like headlines "
      "specifically tailored to the CNN/DailyMail dataset style.")


## Summary

This notebook demonstrates:
1. ✅ Loading a fine-tuned T5-small model for news headline generation
2. ✅ Basic headline generation from news articles  
3. ✅ LangChain integration with HuggingFace pipelines
4. ✅ Multiple headline styles (exciting, formal, short, detailed, neutral)
5. ✅ Interactive headline generation
6. ✅ Comparison with the base T5-small model

### Key Features:
- **Flexible Prompting**: Different styles through temperature and sampling parameters
- **Local Execution**: No API keys required, runs entirely on local hardware
- **Practical NLP**: Real-world application of fine-tuning for summarization
- **Manageable Scale**: Simple project suitable for learning and demonstration

### Next Steps:
- Experiment with different temperature values for various tones
- Try fine-tuning on domain-specific news (sports, tech, etc.)
- Add more sophisticated prompt engineering with LangChain
- Deploy as a web service using Flask or Streamlit
